[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dz-web3/DS-Tech-2026spring/blob/main/Module8_LLM_Finetuning/LLM_Finetuning.ipynb)

**Click the badge above to open this notebook in Google Colab!**

# Module 8: Fine-Tuning Large Language Models (LLMs)

**Data Science for Business (Technical) — Spring 2026**

---

## Learning Objectives

By the end of this module, you will be able to:

1. **Explain** the difference between pre-training, fine-tuning, and prompting
2. **Understand** when fine-tuning is the right choice vs. other approaches
3. **Perform** hands-on fine-tuning using Hugging Face and LoRA
4. **Measure** improvement using before/after evaluation metrics
5. **Evaluate** the business value and trade-offs of fine-tuning LLMs

---

## Why This Matters for Business

Large Language Models like GPT-4, Claude, and Qwen are transforming how businesses operate. But **off-the-shelf models don't always fit your specific needs**. Fine-tuning allows you to:

- 🎯 **Customize** model behavior for your domain (legal, medical, customer service)
- 💰 **Reduce costs** by using smaller, specialized models instead of expensive large ones
- 🔐 **Maintain control** over your data and model behavior
- ⚡ **Improve performance** on specific tasks your business cares about

## 1. Setting Up Google Colab Pro (Free for NYU Students)

### 🎓 NYU Students: Get Free Colab Pro!

Google offers **free Colab Pro subscriptions** for students at U.S. higher education institutions.

**To claim your free subscription:**

1. Go to [colab.research.google.com](https://colab.research.google.com)
2. Click on the gear icon (⚙️) → "Colab Pro"
3. Select "Colab Pro for Education"
4. Verify your student status using your NYU email
5. You'll receive a **1-year free subscription** with more compute resources

### 🖥️ Enabling GPU for This Notebook

Fine-tuning requires a GPU. Here's how to enable it:

1. Go to **Runtime** → **Change runtime type**
2. Set **Hardware accelerator** to **T4 GPU** (or A100 if available with Colab Pro)
3. Click **Save**

Run the cell below to verify GPU is enabled:

In [ ]:
# Check if GPU is available
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU is enabled: {gpu_name}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ GPU is NOT enabled. Please go to Runtime → Change runtime type → Select GPU")

## 2. The LLM Customization Spectrum

Before diving into fine-tuning, let's understand where it fits in the broader landscape of LLM customization:

| Approach | Effort | Data Needed | Use Case |
|----------|--------|-------------|----------|
| **Prompting** | Low | None | Quick tasks, general use |
| **Few-shot Learning** | Low | 5-20 examples | Demonstrate desired format |
| **RAG** (Retrieval) | Medium | Documents | Add knowledge, keep model current |
| **Fine-tuning** | High | 100-10,000+ examples | Change model behavior/style |
| **Pre-training** | Very High | Billions of tokens | Build from scratch (rarely needed) |

### When Should You Fine-Tune?

✅ **Fine-tune when you want to:**
- Change the model's communication style consistently
- Make the model follow specific formats/templates
- Teach domain-specific terminology or behavior
- Improve reliability on repetitive tasks

❌ **Don't fine-tune when you can:**
- Solve the problem with better prompts
- Use RAG to add relevant knowledge
- Use few-shot examples in the prompt

## 3. Understanding Fine-Tuning: The Concept

### Pre-training vs Fine-tuning

**Pre-training** is like giving someone a general education:
- The model learns from massive amounts of text (books, websites, code)
- It learns language patterns, facts, and reasoning
- This is expensive: millions of dollars, weeks of compute time
- Done by companies like Alibaba (Qwen), Meta (Llama), OpenAI (GPT), Google (Gemini)

**Fine-tuning** is like specialized job training:
- Start with a pre-trained model that already "knows" language
- Train it on your specific examples to learn your style/domain
- Much cheaper: can be done in minutes to hours on a single GPU
- This is what **you** can do!

### LoRA: Efficient Fine-Tuning

Traditional fine-tuning updates **all** model parameters — expensive and slow.

**LoRA (Low-Rank Adaptation)** is a clever technique that:
- Freezes the original model weights
- Adds small "adapter" layers that learn your specific task
- Only trains ~1% of the parameters
- Result: **Same quality, 10x less memory, 10x faster!**

Think of it like this: instead of rewriting an entire textbook, you're adding sticky notes with your customizations.

## 4. Hands-On: Fine-Tuning Qwen2.5

Now let's actually fine-tune a model! We'll use:

- **Model**: Qwen2.5-0.5B-Instruct (Alibaba's efficient small model)
- **Library**: Hugging Face Transformers + PEFT (Parameter-Efficient Fine-Tuning)
- **Technique**: LoRA adapters
- **Task**: Create a customer service chatbot

### Step 1: Install Dependencies

This will take about 1-2 minutes. ☕

In [ ]:
%%capture
# Install Hugging Face libraries
!pip install transformers datasets peft accelerate -q

In [ ]:
# Verify installation
import transformers
import peft
print(f"✅ Transformers version: {transformers.__version__}")
print(f"✅ PEFT version: {peft.__version__}")

### Step 2: Load the Base Model (Qwen2.5-0.5B)

We'll load Qwen2.5-0.5B-Instruct, an efficient model from Alibaba that's openly available.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model configuration - Qwen2.5 is openly available, no approval needed!
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

print(f"✅ Model loaded: Qwen2.5-0.5B-Instruct")
print(f"   Parameters: ~0.5 billion")

### Step 3: Prepare the Training Data

We'll use a customer service dataset. In real scenarios, you'd use your own business data.

In [ ]:
# Sample customer service training data
training_data = [
    {"instruction": "What is your return policy?", "response": "Our return policy allows returns within 30 days of purchase with a valid receipt. Items must be in original condition with tags attached."},
    {"instruction": "How do I track my order?", "response": "You can track your order by logging into your account and clicking 'Order History', or use the tracking number from your shipping confirmation email."},
    {"instruction": "Do you offer international shipping?", "response": "Yes, we ship to over 50 countries worldwide. International shipping rates and delivery times vary by destination."},
    {"instruction": "How can I cancel my order?", "response": "To cancel an order, please contact us within 2 hours of placing it. Once an order has been processed for shipping, it cannot be cancelled."},
    {"instruction": "What payment methods do you accept?", "response": "We accept all major credit cards (Visa, Mastercard, Amex), PayPal, Apple Pay, and Google Pay."},
    {"instruction": "How do I reset my password?", "response": "Click 'Forgot Password' on the login page, enter your email, and we'll send you a reset link valid for 24 hours."},
    {"instruction": "Is my personal information secure?", "response": "Yes, we use industry-standard SSL encryption and never share your data with third parties. Your security is our priority."},
    {"instruction": "How do I apply a discount code?", "response": "Enter your discount code in the 'Promo Code' field at checkout and click 'Apply'. The discount will be reflected in your order total."},
    {"instruction": "What are your store hours?", "response": "Our online store is available 24/7. For physical locations, hours vary by store - please check our store locator for specific hours."},
    {"instruction": "How do I contact customer support?", "response": "You can reach us via live chat on our website, email at support@example.com, or call 1-800-EXAMPLE Monday-Friday 9am-6pm EST."},
    {"instruction": "Do you price match?", "response": "Yes, we offer price matching within 14 days of purchase if you find the same item at a lower price from an authorized retailer."},
    {"instruction": "How long does shipping take?", "response": "Standard shipping takes 5-7 business days. Express shipping (2-3 days) and overnight options are also available at checkout."},
    {"instruction": "Can I change my shipping address?", "response": "You can update your shipping address before the order ships by contacting customer support. Once shipped, address changes are not possible."},
    {"instruction": "Do you have a loyalty program?", "response": "Yes! Join our rewards program for free to earn points on every purchase, receive exclusive discounts, and get early access to sales."},
    {"instruction": "What if my item arrives damaged?", "response": "We're sorry to hear that! Please contact us within 48 hours with photos of the damage, and we'll send a replacement or issue a full refund."},
]

print(f"✅ Training data prepared: {len(training_data)} examples")
print(f"\nExample:")
print(f"  Q: {training_data[0]['instruction']}")
print(f"  A: {training_data[0]['response']}")

## 5. 📊 BEFORE Fine-Tuning: Baseline Evaluation

Before we fine-tune, let's see how the **original model** performs on our customer service questions.

This gives us a **baseline** to compare against after fine-tuning.

In [ ]:
def generate_response(model, tokenizer, question, system_prompt="You are a helpful customer service assistant."):
    """Generate a response from the model"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode and extract response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the assistant's response (after the last user message)
    if question in full_response:
        response = full_response.split(question)[-1].strip()
    else:
        response = full_response
    return response[:500]  # Limit length

print("✅ Response generator ready!")

In [ ]:
# Test questions for evaluation
eval_questions = [
    "What's your return policy?",
    "How can I track my package?",
    "Do you ship internationally?",
    "How do I apply a promo code?",
    "My order arrived broken, what should I do?",
]

# Expected keywords for each question (used for simple evaluation)
expected_keywords = [
    ["30 days", "return", "receipt", "original condition"],
    ["track", "order history", "tracking number", "email"],
    ["ship", "international", "countries", "worldwide"],
    ["promo code", "discount", "checkout", "apply"],
    ["48 hours", "photos", "replacement", "refund", "damaged"],
]

print("📋 Evaluation questions prepared!")

In [ ]:
def evaluate_responses(model, tokenizer, questions, expected_keywords):
    """Evaluate model responses and compute metrics"""
    results = []
    total_keywords_found = 0
    total_keywords_expected = 0
    
    for i, question in enumerate(questions):
        response = generate_response(model, tokenizer, question)
        
        # Count how many expected keywords are in the response
        keywords_found = sum(1 for kw in expected_keywords[i] if kw.lower() in response.lower())
        keywords_expected = len(expected_keywords[i])
        
        total_keywords_found += keywords_found
        total_keywords_expected += keywords_expected
        
        results.append({
            "question": question,
            "response": response,
            "keywords_found": keywords_found,
            "keywords_expected": keywords_expected,
            "score": keywords_found / keywords_expected if keywords_expected > 0 else 0
        })
    
    overall_score = total_keywords_found / total_keywords_expected if total_keywords_expected > 0 else 0
    return results, overall_score

print("📊 Evaluating BEFORE fine-tuning...\n")
before_results, before_score = evaluate_responses(model, tokenizer, eval_questions, expected_keywords)

print("=" * 60)
print("BASELINE (Before Fine-Tuning)")
print("=" * 60)
for r in before_results:
    print(f"\nQ: {r['question']}")
    print(f"A: {r['response'][:200]}..." if len(r['response']) > 200 else f"A: {r['response']}")
    print(f"   Keywords found: {r['keywords_found']}/{r['keywords_expected']} ({r['score']:.0%})")

print(f"\n{'='*60}")
print(f"📊 BASELINE SCORE: {before_score:.1%}")
print(f"{'='*60}")

## 6. Add LoRA Adapters and Train

Now let's add LoRA adapters and fine-tune the model on our customer service data.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# LoRA configuration
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Attention layers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA adapters added!")
print(f"   Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
print(f"   Total parameters: {total_params:,}")

In [ ]:
from datasets import Dataset

# Format data for training using Qwen chat template
def format_and_tokenize(example):
    """Format the data using Qwen's chat template"""
    messages = [
        {"role": "system", "content": "You are a helpful customer service assistant. Be friendly, professional, and concise."},
        {"role": "user", "content": example['instruction']},
        {"role": "assistant", "content": example['response']}
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    
    # Tokenize
    tokens = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Create and tokenize dataset
dataset = Dataset.from_list(training_data)
tokenized_dataset = dataset.map(format_and_tokenize, remove_columns=dataset.column_names)

print(f"✅ Dataset formatted and tokenized!")
print(f"   Number of examples: {len(tokenized_dataset)}")

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training configuration
# Each parameter explained:
# - num_train_epochs: How many times to go through all training examples
# - per_device_train_batch_size: Examples processed at once (smaller = less memory)
# - gradient_accumulation_steps: Accumulate gradients over N steps before updating
# - learning_rate: How big each learning step is
# - fp16: Use 16-bit precision (faster, less memory)

training_args = TrainingArguments(
    output_dir="./customer_service_qwen",
    num_train_epochs=10,  # More epochs for better learning
    per_device_train_batch_size=1,  # Small batch size
    gradient_accumulation_steps=2,  # Effective batch size = 1 * 2 = 2
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
    fp16=True,
    warmup_steps=10,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("🚀 Starting training...")
print(f"   Training examples: {len(tokenized_dataset)}")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size} x {training_args.gradient_accumulation_steps} = {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Estimated steps: ~{len(tokenized_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")
print("   Watch the 'loss' value decrease - that means learning is happening!\n")

In [ ]:
# Run training
trainer.train()

print(f"\n✅ Training complete!")

## 7. 📊 AFTER Fine-Tuning: Measure Improvement

Now let's evaluate the **fine-tuned model** and compare to our baseline!

In [ ]:
# Set model to evaluation mode
model.eval()

print("📊 Evaluating AFTER fine-tuning...\n")
after_results, after_score = evaluate_responses(model, tokenizer, eval_questions, expected_keywords)

print("=" * 60)
print("AFTER FINE-TUNING")
print("=" * 60)
for r in after_results:
    print(f"\nQ: {r['question']}")
    print(f"A: {r['response'][:200]}..." if len(r['response']) > 200 else f"A: {r['response']}")
    print(f"   Keywords found: {r['keywords_found']}/{r['keywords_expected']} ({r['score']:.0%})")

print(f"\n{'='*60}")
print(f"📊 AFTER SCORE: {after_score:.1%}")
print(f"{'='*60}")

In [ ]:
# Print comparison
print("\n" + "=" * 60)
print("📈 IMPROVEMENT SUMMARY")
print("=" * 60)
print(f"\n   BEFORE Fine-Tuning: {before_score:.1%}")
print(f"   AFTER Fine-Tuning:  {after_score:.1%}")
print(f"   ───────────────────────")

improvement = after_score - before_score
if improvement > 0:
    print(f"   📈 IMPROVEMENT: +{improvement:.1%}")
    print(f"\n   ✅ Fine-tuning improved the model's responses!")
elif improvement < 0:
    print(f"   📉 Change: {improvement:.1%}")
    print(f"\n   ⚠️ The model may need more training data or epochs.")
else:
    print(f"   → No change")

print("\n" + "=" * 60)

## 8. Test with Your Own Questions

In [ ]:
# Test with NEW questions (not in training data)
new_questions = [
    "Can I get a refund if I don't like the product?",
    "What happens if my package is lost?",
    "Do you have a size guide?",
]

print("🆕 Testing with NEW questions (not in training data):\n")
for q in new_questions:
    response = generate_response(model, tokenizer, q)
    print(f"Customer: {q}")
    print(f"Bot: {response}")
    print("-" * 50)

## 9. Save the Model (Optional)

In [ ]:
# Save the LoRA adapters (small, ~20MB)
model.save_pretrained("customer_service_lora")
tokenizer.save_pretrained("customer_service_lora")
print("✅ Model saved to 'customer_service_lora' folder")

# Check the size
!du -sh customer_service_lora/

## 10. Summary & Key Takeaways

### What We Learned

1. **Fine-tuning** adapts a pre-trained model to your specific needs
2. **LoRA** makes fine-tuning efficient (train only ~1% of parameters)
3. **Evaluation metrics** help you measure if fine-tuning improved performance
4. **Business value** comes from consistency, cost reduction, and customization

### What We Did

- ✅ Loaded Qwen2.5-0.5B (a 0.5 billion parameter model)
- ✅ Established a **baseline** before fine-tuning
- ✅ Added LoRA adapters for efficient training
- ✅ Fine-tuned on customer service data
- ✅ **Measured improvement** with before/after comparison
- ✅ Tested the model on new questions

### Next Steps for Your Career

1. **Experiment**: Try fine-tuning with your own data
2. **Explore**: Look into Hugging Face Hub, OpenAI fine-tuning API
3. **Stay current**: This field moves fast — follow AI news!

---

*Questions? Reach out during office hours or on the course forum.*

## 📝 Required Tasks

Complete the following two task notebooks to practice what you've learned:

### Task 1: Sentiment Fine-Tuning
**Notebook**: `Task1_Sentiment_Finetuning.ipynb`

[![Open Task 1 in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dz-web3/DS-Tech-2026spring/blob/main/Module8_LLM_Finetuning/Task1_Sentiment_Finetuning.ipynb)

In this task, you will:
- Fine-tune a model on product reviews
- Add your own training examples
- Test the model on custom text

---

### Task 2: Prompting vs Fine-Tuning
**Notebook**: `Task2_Prompting_vs_Finetuning.ipynb`

[![Open Task 2 in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dz-web3/DS-Tech-2026spring/blob/main/Module8_LLM_Finetuning/Task2_Prompting_vs_Finetuning.ipynb)

In this task, you will:
- Compare zero-shot vs few-shot prompting
- Improve prompts for better responses
- Create a decision framework for business scenarios